In [1]:
%load_ext autoreload
%autoreload 2

In [49]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set

In [3]:
DAY = 11

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'Monkey 0:\n  Starting items: 57, 58\n  Operation: new = old * 19\n  Test: divisible'
Split Data:
['Monkey 0:',
 '  Starting items: 57, 58',
 '  Operation: new = old * 19',
 '  Test: divisible by 7',
 '    If true: throw to monkey 2',
 '    If false: throw to monkey 3',
 '',
 'Monkey 1:',
 '  Starting items: 66, 52, 59, 79, 94, 73',
 '  Operation: new = old + 1']


In [78]:
test_data = [
"Monkey 0:",
"  Starting items: 79, 98",
"  Operation: new = old * 19",
"  Test: divisible by 23",
"    If true: throw to monkey 2",
"    If false: throw to monkey 3",
"",
"Monkey 1:",
"  Starting items: 54, 65, 75, 74",
"  Operation: new = old + 6",
"  Test: divisible by 19",
"    If true: throw to monkey 2",
"    If false: throw to monkey 0",
"",
"Monkey 2:",
"  Starting items: 79, 60, 97",
"  Operation: new = old * old",
"  Test: divisible by 13",
"    If true: throw to monkey 1",
"    If false: throw to monkey 3",
"",
"Monkey 3:",
"  Starting items: 74",
"  Operation: new = old + 3",
"  Test: divisible by 17",
"    If true: throw to monkey 0",
"    If false: throw to monkey 1",
]

In [7]:
monkey_re = re.compile(r"^Monkey (\d+):")
items_re = re.compile(r"^  Starting items: (.*)")
ops_re = re.compile(r"^  Operation: ([^\s]*)\s+=\s+([^\s]*)\s+([^\s]*)\s+([^\s]*)")
test_re = re.compile(r"^  Test: divisible by (.*)")
true_re = re.compile(r"^    If true: throw to monkey (.*)")
false_re = re.compile(r"^    If false: throw to monkey (.*)")

In [8]:
@dataclass
class Monkey:
    num: int
    items: List[Set[int]]
    operation: List[str]
    test: int
    t: int
    f: int
    inspects: int = 0
    locked_items: List[int] = field(default_factory=list)

In [9]:
word_dict = {'*':"is multiplied by", '+':"increases by" }
op_dict = {'*':operator.mul, '+':operator.add }

In [123]:
def parse(str_data):
    str_iter = iter(str_data)
    monkies = []
    try:
        while True:
            
            monkey = int(monkey_re.match(next(str_iter)).group(1))
            item_strings = items_re.match(next(str_iter)).group(1).split(",")
            items = list(int(n) for n in item_strings if n != "")
            operation = ops_re.match(next(str_iter)).groups()
            test = int(test_re.match(next(str_iter)).group(1))
            t = int(true_re.match(next(str_iter)).group(1))
            f = int(false_re.match(next(str_iter)).group(1))
            monkies.append(Monkey(monkey, items, operation, test, t, f))
            next(str_iter)
    except StopIteration:
        pass
    return monkies

In [128]:
def inspect(m: Monkey, ins_print=False):
    if ins_print: print(f"Monkey {m.num}:")
    for i in m.items:
        if ins_print: print(f"  Monkey inspects an item with a worry level of {i}.")
        try:
            a_old = False
            a = int(m.operation[1])
        except ValueError:
            a_old = True
            a = i
            
        try:
            b = int(m.operation[3])
            b_text = str(b)
        except ValueError:
            b = i
            if (a_old):
                b_text = "itself"
            else:
                b_text = str(b)
        
        new = op_dict[m.operation[2]](a,b)
        if ins_print: print(f"    Worry level {word_dict[m.operation[2]]} {b_text} to {new}.")
    
        new = (new // 3)
        if ins_print: print(f"    Monkey gets bored with item. Worry level is divided by 3 to {new}.")
        
        if (new % m.test) == 0:
            if ins_print: print(f"    Current worry level is divisible by {m.test}.")
            if ins_print: print(f"    Item with worry level {new} is thrown to monkey {m.t}.")
            yield (new, m.t)
        else:
            if ins_print: print(f"    Current worry level is not divisible by {m.test}.")
            if ins_print: print(f"    Item with worry level {new} is thrown to monkey {m.f}.")
            yield (new, m.f)

In [129]:
def round(m_list: List[Monkey]):
    for i in range(len(m_list)):
        for move,target in inspect(m_list[i]):
            m_list[i].inspects += 1
            m_list[target].items.append(move)
        m_list[i].items = []
    return m_list

In [130]:
def simul(str_data, rounds):
    m_list = parse(str_data)
    for i in range(1,rounds+1):
        m_list = round(m_list)
    res = sorted([m.inspects for m in m_list],reverse=True)
    return res[0] * res[1]
    

In [131]:
simul(test_data, 20)

10605

In [132]:
first = simul(data, 20)

In [133]:
first

50830

In [ ]:
submit_answer(DAY, 1, first)

In [79]:
def parse_2(str_data):
    str_iter = iter(str_data)
    monkies = []
    try:
        while True:
            monkey = int(monkey_re.match(next(str_iter)).group(1))
            items = list(int(n) for n in items_re.match(next(str_iter)).group(1).split(","))
            operation = ops_re.match(next(str_iter)).groups()
            test = int(test_re.match(next(str_iter)).group(1))
            t = int(true_re.match(next(str_iter)).group(1))
            f = int(false_re.match(next(str_iter)).group(1))
            monkies.append(Monkey(monkey, items, operation, test, t, f))
            next(str_iter)
    except StopIteration:
        pass
    return monkies

In [109]:
def inspect_2(m: Monkey, total_divisor, ins_print=False):
    if ins_print:
        if m.items:
            print(f"Monkey {m.num}:")
    
    for i in m.items:    
        
        if ins_print: print(f"  Monkey inspects an item with a worry level of {i}.")
        try:
            a_old = False
            a = int(m.operation[1])
        except ValueError:
            a_old = True
            a = i
            
        try:
            b = int(m.operation[3])
            b_text = str(b)
        except ValueError:
            b = i
            if (a_old):
                b_text = "itself"
            else:
                b_text = str(b)
        
        new = op_dict[m.operation[2]](a,b)
        if ins_print: print(f"    Worry level {word_dict[m.operation[2]]} {b_text} to {new}.")

        if (new % m.test) == 0:
            if ins_print: print(f"    Current worry level is divisible by {m.test}.")
            if ins_print: print(f"    Item with worry level {new} is thrown to monkey {m.t}.")
            yield (new % total_divisor, m.t)
        else:
            if ins_print: print(f"    Current worry level is not divisible by {m.test}.")
            if ins_print: print(f"    Item with worry level {new} is thrown to monkey {m.f}.")
            yield (new % total_divisor, m.f)

In [110]:
def round_2(m_list: List[Monkey], total_divisor):
    for i in range(len(m_list)):
        for move,target in inspect_2(m_list[i], total_divisor):
            m_list[i].inspects += 1
            m_list[target].items.append(move)
        m_list[i].items = []
    return m_list

In [116]:
def simul_2(str_data, rounds):
    m_list = parse_2(str_data)
    total_divisor = list(accumulate([m.test for m in m_list], operator.mul))[-1]
    # print(total_divisor)
    for i in range(1,rounds+1):
        m_list = round_2(m_list, total_divisor)
        res = [m.inspects for m in m_list]
    print(res)
    res = sorted(res,reverse=True)
    return res[0] * res[1]

In [121]:
second = simul_2(data, 10000)

[60020, 7592, 119998, 59985, 62507, 57608, 117503, 119999]


In [122]:
submit_answer(DAY, 2, second)

14399640002
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 11! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True